In [1]:
!pip install simhash
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


In [2]:
from simhash import Simhash
import re
from PIL import Image, ImageDraw

def djb2(s: str) -> int:
    _hash = 5381
    for x in s:
        _hash = ((_hash << 5) + _hash) + ord(x)
    return _hash & 0xFFFFFFFF


def hash_str_to_x_y(s):
    x = (s & 0x00FF00) >> 8
    y = s & 0x0000FF
    return x, y


def hash_str_to_color(s):
    r = (s & 0xFF0000) >> 16
    g = (s & 0x00FF00) >> 8
    b = s & 0x0000FF
    return r, g, b

def get_features(s):
    width = 3
    # s = s.lower()
    s = "hello my name is simhash!"
    
    s = re.sub(r'[^\w]+', '', s)
    print(s)
    return [s[i:i + width] for i in range(max(len(s) - width + 1, 1))]

In [3]:
import os

rootPath = '/content/drive/MyDrive/Malware Analysis/'

unpackedBenignBasicBlockDirPath = rootPath + 'kisa_dataset/benign_block/'
unpackedMalignBasicBlockDirPath = rootPath + 'kisa_dataset/malign_block/'

# unpackedBenignImagePath = rootPath + 'kisa_dataset/benign_image_512/'
# unpackedMalignImagePath = rootPath + 'kisa_dataset/malign_image_512/'

unpackedBenignImagePath = rootPath + 'kisa_dataset/benign_image_64/'
unpackedMalignImagePath = rootPath + 'kisa_dataset/malign_image_64/'

In [4]:
def hash_str_to_x_y_r_g_b(s):  # xyrgb_hash = Simhash("".join(block), f=40).value
  x = (s & 0xFF00000000) >> 32
  y = (s & 0x00FF000000) >> 24
  r = (s & 0x0000FF0000) >> 16
  g = (s & 0x000000FF00) >> 8
  b =  s & 0x00000000FF
  return x, y, r, g, b


In [5]:
def hash_str_to_x_y_r_g_b_512(s): # xyrgb_hash = Simhash("".join(block), f=48).value
  x = (s & 0b111111111000000000000000000000000000000000000000) >> 39
  y = (s & 0b000000000111111111000000000000000000000000000000) >> 30
  r = (s & 0b000000000000000000111111110000000000000000000000) >> 22
  g = (s & 0b000000000000000000000000001111111100000000000000) >> 14
  b = (s & 0b000000000000000000000000000000000011111111000000) >> 6
  return x, y, r, g, b


In [6]:
def hash_str_to_x_y_r_g_b_64(s): # xyrgb_hash = Simhash("".join(block), f=40).value
  x = (s & 0b1111111000000000000000000000000000000000) >> 33
  y = (s & 0b0000000111111100000000000000000000000000) >> 26
  r = (s & 0b0000000000000011111111000000000000000000) >> 18
  g = (s & 0b0000000000000000000000111111110000000000) >> 10
  b = (s & 0b0000000000000000000000000000001111111100) >> 2
  return x, y, r, g, b


In [7]:
def bg(xyrgb_list, bg="black"):
  # for 512
  # window = 2**9
  # for 256
  # window = 2**8
  # for 64
  window = 2**7
  img = Image.new("RGB", (window, window), bg)
  draw = ImageDraw.Draw(img)

  
  for line in xyrgb_list:
    x = line[0]
    y = line[1]
    r = line[2]
    g = line[3]
    b = line[4]
    r_, g_, b_ = img.getpixel((x,y))
    draw.point((x,y), fill=(r + r_, g + g_, b + b_))
  

  return img


In [8]:
def make_image(basic_blocks):
  xyrgb_list = []
  
  for block in basic_blocks:
    # xyrgb_hash = Simhash("".join(block), f=48).value
    xyrgb_hash = Simhash("".join(block), f=40).value
    # x, y, r, g, b = hash_str_to_x_y_r_g_b_512(xyrgb_hash)
    x, y, r, g, b = hash_str_to_x_y_r_g_b_64(xyrgb_hash)
    
    xyrgb_list.append([x, y, r, g, b])
  return bg(xyrgb_list)

In [ ]:
#benignBasicBlock to image
benign_basic_block_list = os.listdir(unpackedBenignBasicBlockDirPath)
for benign_basic_block in benign_basic_block_list:
    path = unpackedBenignBasicBlockDirPath + benign_basic_block
    # try:
    basic_block_file = open(path, 'r')
    lines = basic_block_file.readlines()
    basic_blocks = []
    
    for line in lines:
      basic_block = line[:-1].split(',')
      basic_blocks.append(basic_block)
    image = make_image(basic_blocks)
    outfile_name = unpackedBenignImagePath + benign_basic_block[:-4] + '.png'
    image.save(outfile_name)
    
    # except:
      # print(path)

In [ ]:
#malignbasicblock to image
malign_basic_block_list = os.listdir(unpackedMalignBasicBlockDirPath)
print(len(malign_basic_block_list))
for malign_basic_block in malign_basic_block_list:
    path = unpackedMalignBasicBlockDirPath + malign_basic_block
    try:
      basic_block_file = open(path, 'r')
      lines = basic_block_file.readlines()
      basic_blocks = []
      
      for line in lines:
        basic_block = line[:-1].split(',')
        basic_blocks.append(basic_block)
      image = make_image(basic_blocks)
      outfile_name = unpackedMalignImagePath + malign_basic_block[:-4] + '.png'
      image.save(outfile_name)
      
    except:
      print(path)

2699


# New Section

같은 xy좌표애 찍혔을 때 
1. RGB값이 다를 때
-> 더해서 색을 변화시킵니까?
2. RGB값이 같을 때
-> 하얀색으로 가게끔 해서 반복을 반영해야 합니까?
아니면 그냥 같다면 그 상태로 둡니까?



구글 드라이브에 있는 애들하고 연동
앞에있는 3글자만 따오기
베이직 블록 정보들을 저장

불러와서 xyrgb
x - 1 ~ x +1 찍기